# Universidad de Buenos Aires
## Deep Learning - TP1

### _Alumno: Natanael Emir Ferrán_
### _Cohorte: 12_

### Ejercicio 1



Se quiere encontrar el máximo de la siguiente función:

$z = -(x - 2)^2 - (y - 3)^2 + 4$
<br>
<br>
1. Aplicar gradiente de forma analítica e igualar a zero para encontrar los valores de $x$ e $y$ donde $z$ tiene un máximo. Cuál es el valor del máximo?

2. Aplicar SGD para encontrar la ubicación del máximo de manera numérica (pueden utilizar pytorch). Comparar con el resultado obtenido en el punto 1

#### Solución

A fin de obtener los puntos críticos de la función $z$, obtenemos sus derivadas primeras y las igualamos a 0:

$$ \frac{dz}{dx} = -2(x-2) = 0 $$

$$ \frac{dz}{dy} = -2(x-3) = 0 $$

$$ x = 2 $$

$$ y = 3 $$

A raíz de estos resultados, observamos que contamos con un único punto crítico $(2,3)$.

A fin de conocer si se trata de un máximo, un mínimo o un punto de inflexión, armamos la _matriz hessiana_ y analizamos sus autovalores. La misma se conforma de la siguiente manera:

$$ H(z) =
\begin{pmatrix}
\frac{d^{2}z}{dx^{2}} & \frac{d^{2}z}{dxdy} \\
\frac{d^{2}z}{dydx} & \frac{d^{2}z}{dy^{2}}
\end{pmatrix}
$$

Entonces calculamos:

$$ \frac{d^{2}z}{dx^{2}} = -2 $$

$$ \frac{d^{2}z}{dxdy} = 0 $$

$$\frac{d^{2}z}{dydx}=0$$

$$\frac{d^{2}z}{dy^{2}}=-2$$

Reemplazamos y obtenemos:

$$ H(z) =
\begin{pmatrix}
-2 & 0 \\
0 & -2
\end{pmatrix}
$$

Como los autovalores son ambos negativos, entonces concluímos que el punto crítico encontrado se trata de un máximo.

Finalmente, reemplazamos los valores de $x$ y $y$ en la función $z$ a fin de obtener el valor del máximo:

$$
z(x=2,y=3) = -(2-2)^{2}-(3-3)^{2}+4 = 4
$$

***


Seguidamente, utilizaremos `pytorch` para resolver el ejercicio con SGD:

In [1]:
# first we import libraries
import torch
from tqdm import tqdm

In [2]:
# defining function
def function(x, y):
    return -(x - 2)**2 - (y - 3)**2 + 4

# we initialize variables
x = torch.tensor([0.0], requires_grad=True)
y = torch.tensor([0.0], requires_grad=True)

# choosing HP
lr = 0.001
epochs = 5000

# starting loop
for epoch in tqdm(range(epochs)):

    # calculating output
    output = function(x, y)
    
    # getting gradients
    output.backward()
    
    # updating variables
    x.data = x.data + lr * x.grad.data
    y.data = y.data + lr * y.grad.data
    
    # restarting gradients
    x.grad.data.zero_()
    y.grad.data.zero_()

# print results
print("x:", x.item())
print("y:", y.item())
print("Max value obtained:", output.item())

100%|██████████| 5000/5000 [00:00<00:00, 5016.81it/s]

x: 1.9999114274978638
y: 2.999856948852539
Max value obtained: 4.0


### Ejercicio 2


Descargar el dataset del siguiente link: https://drive.google.com/file/d/1eFWn7eDmSFUK1JuuBBykxkC9J0CGYDKe/view?usp=sharing.

El dataset contiene mediciones obtenidas al ensayar un sistema de posicionamiento. El sistema consiste en un dispositivo móvil del cual se desea conocer la posición y 13 "balizas" fijas (distribuidas en un salón) que emiten señales de radio.

Cada fila del dataset contiene una posición del dispositivo móvil y los niveles de señal recibida (de las señales emitidas por cada una de las 13 balizas fijas) en dicha posición.

![Salon](https://drive.google.com/uc?export=view&id=1z3uHEd3tS1kQpGXfhPYn2GFfA95v_ArW)


Algunas consideraciones:
- La imágen anterior es orientativa, no se encuentra a escala ni representa la verdadera posición de las balizas fijas.
- La posición en el salón se divide en una cuadrícula. La posición horizontal se codifica con una letra de la A a la Z y la posición vertical se codifica con valores de 01 a 20.
- El nivel de señal recibida se mide de 0 (máximo teórico) a -200 (mínimo teórico). NA significa que no se recibe señal de la baliza en dicha posición. A efectos prácticos no recibir señal (NA) es equivalente a recibir una señal con nivel -200.

**Consignas:**

1. Analizar el dataset y aplicar las transformaciones que considere necesarias para entrenar un modelo de red neuronal.

2. Entrenar un modelo de **Deep Learning** con múltiples capas lineales que prediga la posición del dispositivo móvil en el salón (vertical y horizontal) a partir de las mediciones de los niveles de las 13 balizas. Graficar la evolución de la función de pérdida y la evolución de la métrica [MAE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html) durante el entrenamiento.

3. Comprobar el funcionamiento del modelo realizando una predicción sobre una muestra aleatoria del dataset y comparar con la posición real.

Con la finalidad de ahorrar energía en el dispositivo móvil y simplificar el sistema, se quiere ensayar la posibilidad de predecir la posición solamente con la información del nivel de señal de las 2 balizas mas cercanas.

4. Aplicar las transformaciones necesarias sobre el dataset para obtener un nuevo dataset que contenga solamente la información de las 2 balizas con mayor nivel de señal (ver imágen adjunta). Si no se recibe señal de una 2da baliza, proponer un método para completar la información faltante.

![Dataset Punto 4](https://drive.google.com/uc?export=view&id=1kz1Y5m5rmbYPiuZIc4QHvnt4uFB2TwWu)


5. Entrenar un modelo de **Deep Learning** que prediga la posición del dispositivo móvil en el salón (vertical y horizontal) a partir del dataset del punto 4, incluyendo **una capa de embeddings** para ambos número (o IDs) de balizas.

6. Comparar los resultados obtenidos con los modelo de los puntos 2 y 5 y enunciar conclusiones.

#### Solución

In [3]:
# first we import essential libraries
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

In [4]:
# reading data
df= pd.read_csv(r'C:\Users\natan\Documents\MEGA\MEGAsync\Posgrado\3er Bimestre\AP\Positioning_data.csv')
df.head()

,Pos,Baliza1,Baliza2,Baliza3,Baliza4,Baliza5,Baliza6,Baliza7,Baliza8,Baliza9,Baliza10,Baliza11,Baliza12,Baliza13
0,O02,NaN,NaN,NaN,NaN,NaN,-78.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P01,NaN,NaN,NaN,NaN,NaN,-78.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P01,NaN,NaN,NaN,NaN,NaN,-77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P01,NaN,NaN,NaN,NaN,NaN,-77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P01,NaN,NaN,NaN,NaN,NaN,-77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# checking basic info
df.shape

(1420, 14)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1420 entries, 0 to 1419
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pos       1420 non-null   object 
 1   Baliza1   25 non-null     float64
 2   Baliza2   497 non-null    float64
 3   Baliza3   280 non-null    float64
 4   Baliza4   402 non-null    float64
 5   Baliza5   247 non-null    float64
 6   Baliza6   287 non-null    float64
 7   Baliza7   50 non-null     float64
 8   Baliza8   91 non-null     float64
 9   Baliza9   31 non-null     float64
 10  Baliza10  29 non-null     float64
 11  Baliza11  25 non-null     float64
 12  Baliza12  35 non-null     float64
 13  Baliza13  44 non-null     float64
dtypes: float64(13), object(1)
memory usage: 155.4+ KB


In [7]:
# checking NaN existence
df.isna().sum(axis=0)

Pos            0
Baliza1     1395
Baliza2      923
Baliza3     1140
Baliza4     1018
Baliza5     1173
Baliza6     1133
Baliza7     1370
Baliza8     1329
Baliza9     1389
Baliza10    1391
Baliza11    1395
Baliza12    1385
Baliza13    1376
dtype: int64

In [8]:
# replacing NaN with -200 
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value= -200.0)
df_transformed= imp.fit_transform(df)
df_transformed= pd.DataFrame(data= df_transformed, columns= df.columns)

df_transformed.head()

,Pos,Baliza1,Baliza2,Baliza3,Baliza4,Baliza5,Baliza6,Baliza7,Baliza8,Baliza9,Baliza10,Baliza11,Baliza12,Baliza13
0,O02,-200.0,-200.0,-200.0,-200.0,-200.0,-78.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
1,P01,-200.0,-200.0,-200.0,-200.0,-200.0,-78.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
2,P01,-200.0,-200.0,-200.0,-200.0,-200.0,-77.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
3,P01,-200.0,-200.0,-200.0,-200.0,-200.0,-77.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
4,P01,-200.0,-200.0,-200.0,-200.0,-200.0,-77.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0


In [9]:
# re-checking NaN existence
df_transformed.isna().sum(axis=0)

Pos         0
Baliza1     0
Baliza2     0
Baliza3     0
Baliza4     0
Baliza5     0
Baliza6     0
Baliza7     0
Baliza8     0
Baliza9     0
Baliza10    0
Baliza11    0
Baliza12    0
Baliza13    0
dtype: int64

In [10]:
# checking data types
df_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1420 entries, 0 to 1419
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Pos       1420 non-null   object
 1   Baliza1   1420 non-null   object
 2   Baliza2   1420 non-null   object
 3   Baliza3   1420 non-null   object
 4   Baliza4   1420 non-null   object
 5   Baliza5   1420 non-null   object
 6   Baliza6   1420 non-null   object
 7   Baliza7   1420 non-null   object
 8   Baliza8   1420 non-null   object
 9   Baliza9   1420 non-null   object
 10  Baliza10  1420 non-null   object
 11  Baliza11  1420 non-null   object
 12  Baliza12  1420 non-null   object
 13  Baliza13  1420 non-null   object
dtypes: object(14)
memory usage: 155.4+ KB


In [11]:
# fixing wrong data types
df_transformed[['Baliza1', 'Baliza2', 'Baliza3', 'Baliza4', 'Baliza5', 'Baliza6','Baliza7', 'Baliza8', 'Baliza9', 'Baliza10', 'Baliza11', 'Baliza12','Baliza13']] = df_transformed[['Baliza1', 'Baliza2', 'Baliza3', 'Baliza4', 'Baliza5', 'Baliza6','Baliza7', 'Baliza8', 'Baliza9', 'Baliza10', 'Baliza11', 'Baliza12','Baliza13']].apply(pd.to_numeric)

In [12]:
# re-checking data types
df_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1420 entries, 0 to 1419
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pos       1420 non-null   object 
 1   Baliza1   1420 non-null   float64
 2   Baliza2   1420 non-null   float64
 3   Baliza3   1420 non-null   float64
 4   Baliza4   1420 non-null   float64
 5   Baliza5   1420 non-null   float64
 6   Baliza6   1420 non-null   float64
 7   Baliza7   1420 non-null   float64
 8   Baliza8   1420 non-null   float64
 9   Baliza9   1420 non-null   float64
 10  Baliza10  1420 non-null   float64
 11  Baliza11  1420 non-null   float64
 12  Baliza12  1420 non-null   float64
 13  Baliza13  1420 non-null   float64
dtypes: float64(13), object(1)
memory usage: 155.4+ KB


In [13]:
# checking quantity of 'Pos' unique values
len(df_transformed.Pos.unique())

105

In [14]:
# we have too many labels, so using OHE will increase dimensionality considerably.
# even though, we are trying if it works well
# if not, we are using an embedding layer instead

In [15]:
# splitting data
from sklearn.model_selection import train_test_split

x= df_transformed.drop(['Pos'], axis= 1).values
y= pd.get_dummies(df['Pos']).values

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size= 0.3, stratify= y)

In [16]:
# creating dataset in order to work with pytorch
class MyDataset(Dataset):

  def __init__(self, x, y):
    self.x = x
    self.y = y

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [17]:
# creating train and test sets
train_ds= MyDataset(X_train, y_train)
test_ds= MyDataset(X_test, y_test)

In [18]:
# checking getitem correct functionality
train_ds[20]

(array([-200.,  -71., -200., -200.,  -69., -200., -200., -200., -200.,
        -200., -200., -200., -200.]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8))

In [19]:
test_ds[20]

(array([-200.,  -72., -200., -200.,  -69., -200., -200., -200., -200.,
        -200., -200., -200., -200.]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8))

In [20]:
# checking len correct functionality
len(train_ds)

994

In [21]:
len(test_ds)

426

In [22]:
# creating batches
train_dataloader = DataLoader(train_ds, batch_size = 64, shuffle= True)
test_dataloader = DataLoader(test_ds, batch_size= 64)

In [23]:
# creating NN class
class NNet(torch.nn.Module):

  def __init__(self):
    # defining NN architecture
    super().__init__()
    self.linear_1= torch.nn.Linear(in_features= 13, out_features= 200, bias= True)
    self.relu_1= torch.nn.ReLU()
    self.linear_2= torch.nn.Linear(in_features= 200, out_features= 200, bias= True)
    self.relu_2= torch.nn.ReLU()
    self.linear_3= torch.nn.Linear(in_features= 200, out_features= 105, bias= True)

  def forward(self, x):
    # defining forward propagation calculation
    x= self.linear_1(x)
    x= self.relu_1(x)
    x= self.linear_2(x)
    x= self.relu_2(x)
    x= self.linear_3(x)

    return x

In [24]:
# initialize and check NN schema
nnet= NNet()
print(nnet)

NNet(
  (linear_1): Linear(in_features=13, out_features=200, bias=True)
  (relu_1): ReLU()
  (linear_2): Linear(in_features=200, out_features=200, bias=True)
  (relu_2): ReLU()
  (linear_3): Linear(in_features=200, out_features=105, bias=True)
)


In [25]:
# checking quantity of trainable parameters
print(sum(p.numel() for p in nnet.parameters()))

64105


In [26]:
# choosing loss function and optimizer
loss_function= torch.nn.CrossEntropyLoss(reduction= 'mean')
optimizer= torch.optim.Adam(nnet.parameters(), lr= 0.01)

In [27]:
# using gpu if available
device= "cpu"
if torch.cuda.is_available():
  device= "cuda:0"

device

'cpu'

In [28]:
# mounting NN in device 
nnet= nnet.to(device)

In [29]:
# epochs quantity
epochs= 10

train_loss_by_epoch= []
test_loss_by_epoch= []

# Mini-Batch training
for epoch in range(epochs):
  
  ############################################
  ## Training
  ############################################
  nnet.train(True)

  epoch_loss= 0
  epoch_y_hat= []
  epoch_y= []
  
  for i,data in enumerate(train_dataloader):
    # getting data from training batches
    x_batch, y_batch= data
    # copying data to device
    x_batch= x_batch.to(device).float()
    y_batch= y_batch.to(device).float().reshape(-1, 1)

    #### forward propagation ####
    # restarting gradients
    optimizer.zero_grad()
    nnet_output= nnet(x_batch)
    y_batch_hat= torch.softmax(nnet_output)
    
    # Loss calculation
    loss= loss_function(nnet_output, y_batch)

    #### backpropagation ####
    loss.backward()

    # updating parameters
    optimizer.step()

    # save true and predicted values in order to calculate metrics
    epoch_y += list(y_batch.detach().cpu().numpy())
    epoch_y_hat += list(y_batch_hat.detach().cpu().numpy())
    # save batch-loss
    epoch_loss = epoch_loss + loss.item()

  # loss-mean
  epoch_loss = epoch_loss / n_train
  # save epoch-loss for graphs
  train_loss_by_epoch.append(epoch_loss)
  # calculate metric of the epoch
  accuracy = metrics.accuracy_score(epoch_y, [j>=0.5 for j in epoch_y_hat])

  ############################################
  ## Testing
  ############################################
  # disable gradient calculation
  nnet.train(False)

  valid_epoch_loss = 0
  valid_epoch_y_hat = []
  valid_epoch_y = []

  for i,data in enumerate(valid_dataloader):
    # getting data from testing batches
    x_batch, y_batch= data
    # copying data to device
    x_batch= x_batch.to(device).float()
    y_batch= y_batch.to(device).float().reshape(-1, 1)

    #### forward propagation ####
    nnet_output= nnet(x_batch)
    y_batch_hat= torch.softmax(nnet_output)
    
    # Loss calculation
    loss= loss_function(nnet_output, y_batch)

    # save true and predicted values in order to calculate metrics
    valid_epoch_y += list(y_batch.detach().cpu().numpy())
    valid_epoch_y_hat += list(y_batch_hat.detach().cpu().numpy())
    # save batch-loss
    valid_epoch_loss = valid_epoch_loss + loss.item()

  # loss-mean
  valid_epoch_loss = valid_epoch_loss / n_valid
  # save epoch-loss for graphs
  valid_loss_by_epoch.append(valid_epoch_loss)
  # calculate metric of the epoch
  valid_accuracy = metrics.accuracy_score(valid_epoch_y, [j>=0.5 for j in valid_epoch_y_hat])

  ############################################
  ### print results per epoch
  ############################################
  print(f" Epoch {epoch} | " \
        f"Train/Valid loss: {epoch_loss:.3f} / {valid_epoch_loss:.3f} | " \
        f"Train/Valid accuracy: {accuracy:.3f} / {valid_accuracy:.3f}")

TypeError: softmax() received an invalid combination of arguments - got (Tensor), but expected one of:
 * (Tensor input, int dim, torch.dtype dtype, *, Tensor out)
 * (Tensor input, name dim, *, torch.dtype dtype)


In [ ]:
fig, ax1= plt.subplots(1,1)
ax1.plot(train_loss_by_epoch, label="Train loss")
ax1.plot(valid_loss_by_epoch, label="Test loss")
ax1.legend()
ax1.set_title("Loss by epoch")
ax1.set_xlabel("epoch")
ax1.set_ylabel("CE")